In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
json_prepared = dataiku.Dataset("Json_prepared")
df = json_prepared.get_dataframe()




In [2]:
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import time

import os

In [4]:
df.head()

,Timestamp,date,broker-102/,broker-104/,broker-106/,broker-108/,broker-110/,zook-102/,zook-104/,broker-102/data,broker-104/data,broker-106/data,broker-108/data,broker-110/data,zook-102/data,zook-104/data,broker-101/,broker-103/,broker-105/,broker-107/,broker-109/,zook-101/,zook-103/,zook-105/,broker-101/data,broker-103/data,broker-105/data,broker-107/data,broker-109/data,zook-101/data,zook-103/data,zook-105/data
0,1624601700,2021-06-25 06:15:00+00:00,16.318256,14.317784,15.499574,13.478442,10.100069,3.457606,3.497103,85.198139,74.491702,70.937154,73.322070,88.858424,0.278296,0.216338,15.759843,15.526538,10.510089,13.464054,9.834396,3.489473,3.470773,3.176120,81.194161,76.236908,73.175427,72.768512,90.265277,0.251977,0.236909,0.245222
1,1624602600,2021-06-25 06:30:00+00:00,16.201163,14.317455,15.596986,13.478452,10.100073,3.457596,3.498734,85.195519,74.540221,70.971025,73.390322,88.953916,0.278296,0.216338,15.798915,15.604688,10.588215,13.542199,9.893027,3.469877,3.470783,3.176225,81.216755,76.227287,73.233565,72.715046,90.343407,0.251977,0.236909,0.245222
2,1624603500,2021-06-25 06:45:00+00:00,16.240144,14.200271,15.479296,13.478466,10.021952,3.457639,3.498534,85.368338,74.687954,71.060131,73.450625,89.056802,0.278296,0.216338,15.875768,15.526571,10.480893,13.464078,9.971639,3.489372,3.470907,3.156653,81.287027,76.350551,73.217323,72.744789,90.583015,0.251977,0.236909,0.245222
3,1624604400,2021-06-25 07:00:00+00:00,16.166954,14.239343,15.479368,13.351567,10.002428,3.458865,3.498543,85.503684,74.622017,71.028285,73.530478,89.180835,0.278296,0.216338,15.875782,15.487523,10.519952,13.464083,9.971339,3.470134,3.470964,3.176167,81.417682,76.318796,73.396165,72.921757,90.711837,0.251977,0.236909,0.245222
4,1624605300,2021-06-25 07:15:00+00:00,16.240354,14.317498,15.597000,13.405028,10.002437,3.497909,3.499302,85.652764,74.762854,71.062395,73.475029,89.232812,0.278296,0.216338,15.758903,15.458370,10.446685,13.542228,9.893218,3.470191,3.472156,3.176158,81.569016,76.359671,73.456350,72.810311,90.751683,0.251977,0.236909,0.245222


In [12]:
del df['Timestamp']

In [24]:

# Split into training, validation and test datasets.
# Since it's timeseries we should do it by date.
test_cutoff_date = df['date'].max() - timedelta(days=7)

test_df = df[df['date'] > test_cutoff_date]
train_df = df[df['date'] <= test_cutoff_date]

#check out the datasets
print('Test dates: {} to {}'.format(test_df['date'].min(), test_df['date'].max()))
print('Train dates: {} to {}'.format(train_df['date'].min(), train_df['date'].max()))

Test dates: 2021-07-29 13:15:00+00:00 to 2021-08-05 13:00:00+00:00
Train dates: 2021-06-25 06:15:00+00:00 to 2021-07-29 13:00:00+00:00


In [25]:
train_df.set_index('date')
test_df.set_index('date')

,broker-102/,broker-104/,broker-106/,broker-108/,broker-110/,zook-102/,zook-104/,broker-102/data,broker-104/data,broker-106/data,broker-108/data,broker-110/data,zook-102/data,zook-104/data,broker-101/,broker-103/,broker-105/,broker-107/,broker-109/,zook-101/,zook-103/,zook-105/,broker-101/data,broker-103/data,broker-105/data,broker-107/data,broker-109/data,zook-101/data,zook-103/data,zook-105/data
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-07-29 13:15:00+00:00,13.720007,12.333150,13.528279,11.297396,7.427099,3.430461,3.476887,80.043514,74.903277,70.981028,63.937553,79.504916,0.210270,0.248832,13.848594,13.314026,8.847573,11.026321,7.791537,3.433699,3.456452,3.139422,80.246912,71.995625,65.319841,67.361814,84.180543,0.288578,0.240634,0.218101
2021-07-29 13:30:00+00:00,13.719845,12.329368,13.600863,11.375541,7.505573,3.430489,3.476968,80.150853,75.059778,70.969254,63.998439,79.615596,0.210270,0.248832,13.921994,13.236076,8.964815,10.908788,7.791547,3.433732,3.456118,3.139503,80.290354,72.120999,65.560823,67.395015,84.254019,0.288578,0.240634,0.218101
2021-07-29 13:45:00+00:00,13.798009,12.177808,13.465266,11.253850,7.446828,3.425615,3.477054,80.259573,75.051307,71.167891,64.033788,79.775955,0.210270,0.248832,13.889054,13.274691,8.886736,10.798314,7.791561,3.433718,3.456428,3.139541,80.249784,72.225875,65.620122,67.558497,84.268543,0.288585,0.240642,0.218109
2021-07-29 14:00:00+00:00,13.719874,12.182554,13.600562,11.293476,7.524787,3.425625,3.477064,80.400536,75.143014,71.297558,64.140316,79.771133,0.210270,0.248832,14.019845,13.274453,8.828149,10.801986,7.869697,3.435177,3.456361,3.139594,80.470778,72.359056,65.598920,67.546362,84.367720,0.288585,0.240642,0.218109
2021-07-29 14:15:00+00:00,13.719893,12.348673,13.626668,11.371411,7.510084,3.425873,3.479749,80.517206,75.251217,71.385486,64.202720,79.882102,0.210270,0.248832,14.019535,13.352617,8.964924,10.807108,7.869716,3.435201,3.457096,3.139627,80.618745,72.312251,65.775230,67.603474,84.505704,0.288585,0.240642,0.218109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-05 12:00:00+00:00,13.814648,12.367258,13.324847,11.211664,7.425907,3.432297,3.501834,81.183556,75.722181,72.442575,64.680612,80.087917,0.199385,0.271480,13.714165,13.213457,9.054157,10.901859,7.734948,3.439756,3.471694,3.160997,80.622212,73.903582,65.366397,68.339098,85.784478,0.282800,0.271495,0.192355
2021-08-05 12:15:00+00:00,13.830934,12.405968,13.404136,11.073739,7.365917,3.429459,3.503179,81.119728,75.695875,72.504412,64.723052,80.318505,0.199385,0.271480,13.713664,13.291707,8.963041,10.901573,7.683562,3.439765,3.467158,3.160864,80.549015,73.917483,65.516772,68.476230,85.871186,0.282800,0.271495,0.192362
2021-08-05 12:30:00+00:00,13.774250,12.484008,13.343064,11.132350,7.512431,3.429569,3.503026,81.238667,75.671376,72.502473,64.830883,80.548620,0.199385,0.271480,13.801967,13.154812,9.040580,10.972388,7.761702,3.438644,3.472042,3.160945,80.573072,73.829986,65.635207,68.621475,86.066088,0.282800,0.271495,0.192362


In [32]:
train_df.head()

,date,broker-102/,broker-104/,broker-106/,broker-108/,broker-110/,zook-102/,zook-104/,broker-102/data,broker-104/data,broker-106/data,broker-108/data,broker-110/data,zook-102/data,zook-104/data,broker-101/,broker-103/,broker-105/,broker-107/,broker-109/,zook-101/,zook-103/,zook-105/,broker-101/data,broker-103/data,broker-105/data,broker-107/data,broker-109/data,zook-101/data,zook-103/data,zook-105/data
0,-1.000000,0.758251,0.804179,0.917667,0.798042,0.750249,-0.013011,0.155489,0.644533,0.512796,0.283273,0.514682,0.483258,0.105025,-1.0,0.803128,0.800177,0.697282,0.775338,0.677041,0.220878,0.103615,0.162571,0.595796,0.496005,0.822820,0.173068,0.457156,-0.563708,-0.627823,-0.228127
1,-0.999392,0.683900,0.803939,0.997461,0.798048,0.750252,-0.013157,0.186505,0.644295,0.518380,0.287806,0.521555,0.489407,0.105025,-1.0,0.832987,0.855010,0.767237,0.825314,0.719698,-0.056037,0.103798,0.164734,0.598186,0.494777,0.828691,0.168355,0.464394,-0.563708,-0.627823,-0.228127
2,-0.998785,0.708652,0.718453,0.901057,0.798058,0.702592,-0.012498,0.182696,0.659975,0.535386,0.299730,0.527627,0.496032,0.105025,-1.0,0.891717,0.800201,0.671140,0.775353,0.776894,0.219463,0.106171,-0.238648,0.605620,0.510509,0.827051,0.170977,0.486593,-0.563708,-0.627823,-0.228127
3,-0.998177,0.662179,0.746956,0.901115,0.708614,0.690680,0.006340,0.182878,0.672256,0.527796,0.295469,0.535668,0.504019,0.105025,-1.0,0.891727,0.772803,0.706113,0.775356,0.776675,-0.052397,0.107267,0.163554,0.619441,0.506457,0.845110,0.186577,0.498528,-0.563708,-0.627823,-0.228127
4,-0.997569,0.708786,0.803971,0.997473,0.746296,0.690686,0.606450,0.197297,0.685782,0.544007,0.300033,0.530084,0.507366,0.105025,-1.0,0.802410,0.752348,0.640509,0.825333,0.719837,-0.051589,0.130089,0.163358,0.635450,0.511674,0.851188,0.176753,0.502219,-0.563708,-0.627823,-0.228127


In [26]:
train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in test_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

/opt/dataiku/code-env/lib64/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype datetime64[ns] was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/dataiku/code-env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/dataiku/code-env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/dataiku/code-env/lib/python3.6/s

In [27]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations  
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [30]:
n_past = 10
n_future = 5 
n_features = 30

In [31]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))


ValueError: cannot reshape array of size 1016180 into shape (3278,10,30)

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

predictions_df = json_prepared_df # For this sample code, simply copy input to output


# Write recipe outputs
predictions = dataiku.Dataset("predictions")
predictions.write_with_schema(predictions_df)